In [1]:
# data manipulation imports
import numpy as np
import pandas as pd

# data saving imports
import pickle
import os

# custom imports
from regression_class import RedditRegression as RR
from regression_class import TimestampClass

In [2]:
# the other 26 file (.py) must be run to run all required regressions


# infiles
regression_infile = "regression_thread_data.p"
thread_infile = 'clean_5_thread_data.p'

# outfiles
outdir = 'logistic_regression/logregs_23102023'
metrics_outfile = "regression_metrics"

# script output file
script_infile = f"{outdir}/logregs_pickle.p"

In [ ]:
script_output = pickle.load(open(script_infile, 'rb'))
subreddit_logregs = script_output['logregs']
regression_params_dict = script_output['regression_params']
out_params_dict = script_output["out_params"]
outdir_names = script_output["subdirs"]